In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Conv2D
from keras.layers import LSTM, Reshape

Using TensorFlow backend.


In [2]:
train_data = "/home/piyush/Documents/IEMOCAP/test/small_train"
test_data = "/home/piyush/Documents/IEMOCAP/test/small_test"

In [6]:
TimeDistributed?

In [7]:
model = Sequential()
# define CNN model
model.add(TimeDistributed(Conv2D(32, (3, 3), activation = 'relu', input_shape = (239,720, 1)),input_shape = (32, 239,720, 1)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
model.add(TimeDistributed(Flatten()))
# # define LSTM model
# model.add(TimeDistributed(Reshape((1355584,))))
model.add(LSTM(128, return_sequences = False))
# model.add(Dense(128, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 32, 237, 718, 32)  320       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 118, 359, 32)  0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 32, 1355584)       0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               694125056 
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 694,126,021
Trainable params: 694,126,021
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model = Sequential()
# define CNN model
model.add(Conv2D(1, (3, 3), activation = 'relu' ,input_shape = (239,720,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
# # define LSTM model
# model.add(TimeDistributed(Reshape((1355584,))))
# model.add(LSTM(128, return_sequences = False))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 237, 718, 1)       10        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 118, 359, 1)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 42362)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               5422464   
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645       
Total params: 5,423,119
Trainable params: 5,423,119
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
model = Sequential()
# define CNN model
model.add(TimeDistributed(Conv2D(5, (3, 3), activation = 'relu'),input_shape = (5, 239,720,1)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
model.add(TimeDistributed(Flatten()))
# # define LSTM model
# model.add(TimeDistributed(Reshape((1355584,))))
# model.add(LSTM(128, return_sequences = False))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_4 (TimeDist (None, 5, 237, 718, 5)    50        
_________________________________________________________________
time_distributed_5 (TimeDist (None, 5, 118, 359, 5)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 5, 211810)         0         
_________________________________________________________________
dense_6 (Dense)              (None, 5, 128)            27111808  
_________________________________________________________________
dense_7 (Dense)              (None, 5, 5)              645       
Total params: 27,112,503
Trainable params: 27,112,503
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
# Finding minumum number of frames in any video
import os
import subprocess

datafile_loc = "Documents/IEMOCAP/EmoEvaluation/"
original_vid_loc = "Documents/IEMOCAP/avi/DivX/"
all_vid_names = subprocess.check_output(["ls",original_vid_loc]).split()
all_names = [x[:-4] for x in all_vid_names]
min_time = float("inf")

for name in all_names:
    txt_file_name = name + '.txt'
    with open(datafile_loc + txt_file_name) as f:
        data = f.readlines()
    data = iter(data)
    try:
        for line in data:
            if line != '\n':
                continue;
            line = next(data)
            start_time = line.split()[0][1:]
            end_time = line.split()[2][:-1]
#             print  float(end_time)-float(start_time)3
            min_time = min(min_time, float(end_time)-float(start_time))
    except(StopIteration):
        None
        
max_frames = int(min_time*30)
print max_frames

25


In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data,
                                                    target_size=(239, 720),
                                                    batch_size=5,
                                                    class_mode='categorical',
                                                    color_mode = 'grayscale')

validation_generator = test_datagen.flow_from_directory(test_data,
                                                        target_size=(239, 720),
                                                        batch_size=5,
                                                        class_mode='categorical',
                                                        color_mode='grayscale')

Found 1575 images belonging to 5 classes.
Found 401 images belonging to 5 classes.


In [37]:
import matplotlib.pyplot as plt
for i in train_generator:
#     plt.imshow(i[0][0], cmap='gray')
#     plt.xticks([]),plt.yticks([])  # to hide tick values on X and Y axis
#     plt.show()
    print i[0][0][0][0]
    break
print type(train_generator)

[0.01176471]
<class 'keras_preprocessing.image.DirectoryIterator'>


In [15]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

def extract_videos_for_conv2d(video_input_file_path, feature_output_file_path, max_frames):
    if feature_output_file_path is not None:
        if os.path.exists(feature_output_file_path):
            return np.load(feature_output_file_path)
    count = 0
    print('Extracting frames from video: ', video_input_file_path)
    vidcap = cv2.VideoCapture(video_input_file_path)
    success, image = vidcap.read()
    features = []
    success = True
    while success and count < max_frames:
#         plt.imshow(image, cmap='gray')
#         plt.xticks([]),plt.yticks([])  # to hide tick values on X and Y axis
#         plt.show()
#         vidcap.set(cv2.CAP_PROP_POS_MSEC, (count * 1000))  # added this line
        success, image = vidcap.read()
#         print('Read a new frame: ', success)
        if success:
#             image = cv2.resize(image, (240, 240), interpolation=cv2.INTER_AREA)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#             channels = image.shape[2]
            image = np.reshape(image, (480,720,1))
#             print image.shape
#             for channel in range(channels):
#                 features.append(image[:, :, channel])
#             count = count + 1
            features.append(image)
            count = count+1
    unscaled_features = np.array(features)
    print unscaled_features.shape
    unscaled_features = np.transpose(unscaled_features, axes=(1, 2, 3, 0))
    print(unscaled_features.shape)
    if feature_output_file_path is not None:
        np.save(feature_output_file_path, unscaled_features)
    return unscaled_features

In [16]:
vid = "/home/piyush/Documents/IEMOCAP/test2/Ses01F_impro01_F000.avi"
out = "/home/piyush/Documents/IEMOCAP/test2/out"
feat = extract_videos_for_conv2d(vid, out, max_frames)
new_feat = np.reshape(feat, (480,720, 1, max_frames))
x_sample = []
x_sample.append(new_feat)
x_sample.append(new_feat)
print len(x_sample)

('Extracting frames from video: ', '/home/piyush/Documents/IEMOCAP/test2/Ses01F_impro01_F000.avi')
(25, 480, 720, 1)
(480, 720, 1, 25)
2


In [ ]:
('Extracting frames from video: ', '/home/piyush/Documents/IEMOCAP/test2/Ses01F_impro01_F000.avi')
(25, 480, 720, 1)
(480, 720, 1, 25)
2

In [11]:
# Ouput when using simple CNN
model.fit_generator(
        train_generator,
        steps_per_epoch=1575,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=401)

Epoch 1/1
1575/1575 [==============================] - 559s 355ms/step - loss: 4.9132 - acc: 0.6946 - val_loss: 12.0258 - val_acc: 0.2539


In [125]:
# CNN with timedistributed layer
model.fit_generator(
        train_generator,
        steps_per_epoch=1575,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=401)

Epoch 1/1


ValueError: could not broadcast input array from shape (239,720,1) into shape (239,720,1,1)